In [1]:
# Tensorflow Version 1.15 
# Keras Version 2.2.4
# Python Version 3.6 

import scipy
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from keras.layers import Dense
import numpy as np

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\py36\lib\site-packag

In [2]:
# import shuffled dataset. The dataset combined Mashup and Morgan fingerprint. 
# t1-t14: labels, marked in "0" or "1" 
# f1-f512: fingerprint, marked in "0" or "1" 
# f513-f912: Mashup vector 

data, meta = scipy.io.arff.loadarff('512_400_S.arff')
df = pd.DataFrame(data)

In [3]:
#df

In [4]:
# Set 1-14 col as labels, set 15-926 col as feature vectors

X = df.iloc[:,14:926].values
y = df.iloc[:,0:14].values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#y_train = y_train.astype(np.float64)
#y_test = y_test.astype(np.float64)

In [5]:
# Load the model and predit. 
# 10-Cross validation, 10 models in total. 
# If you use cuda to run it, the minimun graph card memory is 4GB. 

for i in range(10):
    from keras.models import load_model
    model = load_model('my_model'+str(i)+'.h5')
    result_array = model.predict(X[0:3884])   # Prediction
    np.savetxt("./K-Cross[%s].txt"%i, result_array,fmt='%f',delimiter=',')   # Save to the txt

W0301 17:15:36.247568 13972 deprecation_wrapper.py:119] From C:\Anaconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0301 17:15:36.267565 13972 deprecation_wrapper.py:119] From C:\Anaconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0301 17:15:36.390564 13972 deprecation_wrapper.py:119] From C:\Anaconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0301 17:15:36.392563 13972 deprecation_wrapper.py:119] From C:\Anaconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0301 17:15:36.392563 13972 deprecation_wrapper.py:119] From C:\Anaconda3\e

In [8]:
# Judgment threshold setting 
# There are three regulations: 
# 1. find all items that are over 0.6. 
# 2. find the max item. 
# 3. find all items that are less than 0.1 compared to the max item. 

for m in range(10):
    with open('./K-Cross[%s].txt'%m, 'r') as f:
        for j in range(3883):
            content = f.readline()   # Read prediction vector line
            #print(content)
            content = content.strip('\n') 
            
            predict = list(map(float, content.strip().split(',')))    # Split vector items into list
            max = np.max(predict)   # Find max item
            max_location = np.argmax(predict)   # Find max item locaton
            result = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]   # Generate empty list to store prediction result (0 or 1).
            result[(max_location)] = 1   # Mark max item as 1
            zero = 0   # Number of 0 in the list
            for i in range(14):
                if predict[i] > max-0.1:   # Find all items that are less than 0.1 compared to the max item. 
                    result[i] = 1
            for i in range(14):
                if predict[i] >= 0.6:   # Find all items that are over 0.6. 
                    result[i] = 1
            for i in range(14):
                zero = zero + int(result[i])
            if zero == 14:                
                for k in range(14):
                    result[k] = 0
                result[(max_location)] = 1
            #result = ','.join(result) 

            # text processing
            result = str(result)
            result = result.replace(" ", "")
            result = result.replace("[", "")
            result = result.replace("]", "")
            # Write into txt
            with open('K-cross_results[%s].txt'%m,'a') as file_handle:
                file_handle.write(result)  
                file_handle.write('\n') 



In [1]:
# Generate "Abs True" and "Not Abs True (False)" text for statistics

for m in range (10):
    file1path = 'atc-label-512-400_S.txt'
    file2path = 'K-cross_results[%s].txt'%m
    file_1 = open(file1path,'r')
    file_2 = open(file2path,'r')
    j = 0
    with open('results_stat[%s].txt'%m,'a') as result_stat:
        for i in range(3883):
            m = file_1.readline()
            n = file_2.readline()
            if m==n:
                j += 1
                result_stat.write('Abs True')
                result_stat.write('\n')
            else:
                result_stat.write('False')
                result_stat.write('\n')
    #print(j)